In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
df = pds.random_data(size=2000, n_cols=0).select(
    pds.random(0.0, 1.0).alias("x1"),
    pds.random(0.0, 1.0).alias("x2"),
    pds.random(0.0, 1.0).alias("x3"),
    pds.random_int(0, 3).cast(pl.String).alias("str"),
    pds.random_int(0, 2).alias("target"),
)
df

In [ ]:
import polars_ds.transforms as t

df1 = df.select(
    t.woe_encode(df, ["str"], target = "target")
)
df1

In [ ]:
from category_encoders import WOEEncoder

X = df["str"].to_numpy()
y = df["target"].to_numpy()

enc = WOEEncoder()
df2 = pl.from_pandas(enc.fit_transform(X, y))
df2.columns = ["str"]
df2

In [ ]:
df.select(
    pds.query_woe_discrete("str", "target")
).unnest('str')

In [ ]:
df.select(
    pds.query_iv_discrete("str", "target", return_sum = False)
).unnest("str")

In [ ]:
df.select(
    pds.query_iv("str", "target", return_sum = False)
)

In [ ]:
df.select(
    pl.col("str")
)